In [1]:
# Import files
import moviepy.editor as mp 
import wave
import numpy as np

In [2]:
word_size = 16 # Size of the number in byte format
secret_key = 567 # Secret key already known by the program 

### Encryption

In [3]:
# Load video files and change it into the wav format
video = mp.VideoFileClip("Video.mp4") 
audio = video.audio
audio.write_audiofile("Audio.wav")
audio = wave.open("Audio.wav", mode='rb')

chunk:   0%|                                                                        | 0/2986 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in Try.wav


MoviePy - Done.


In [4]:
# Load bytes from the audio file
frame_data = audio.readframes(audio.getnframes())
frame_bytes = bytearray(list(frame_data))

In [5]:
# Frame data to be written first element is number of frame object
frame_data = [4,165,175,200,1000]
number_element = frame_data[0]

In [6]:
# Convert the frame into bit data of word size and then store it in the audio file
frame_data = ''.join(list(map(lambda number: '{0:0{width}b}'.format(number,width=word_size),frame_data)))
frame_data = list(map(int,frame_data))

In [7]:
# Peform steganography encrpt the number of frames in first bytes
for i, bit in enumerate(frame_data[:word_size]):
    frame_bytes[i] = (frame_bytes[i] & 254) | bit

In [8]:
# Get modified audio file size and divide it into sample sizes
frame_bytes_length = len(frame_bytes)-word_size
frame_data = frame_data[word_size:]
sample_size = frame_bytes_length//len(frame_data)

In [9]:
# Arrange the bits in random order
random_position = np.random.RandomState(seed=number_element*secret_key).permutation(number_element*word_size)
random_dict = zip(random_position,frame_data)
random_dict = sorted(random_dict,key = lambda x:x[0])
frame_data = [i[1] for i in random_dict]

In [10]:
# Peform steganography and save the data in the encrypted format
for i, bit in enumerate(frame_data):
    temp_frame_data = frame_bytes[word_size+i*sample_size:word_size+(i+1)*sample_size]
    
    # Get parity by xor the number 
    parity = 0 
    for frame in temp_frame_data:
        parity = parity^frame
    
    # Xor each bit to generate final parity number
    final_parity = 0 
    for parity_bit in '{0:08b}'.format(parity):
        final_parity = final_parity^int(parity_bit)
        
    if final_parity != bit:
        frame_bytes[word_size+(i+1)*sample_size-1] = (frame_bytes[word_size+(i+1)*sample_size-1] ^ 1) # Reverse the bit if parity not equal to the bit to be encoded

In [11]:
# Generate modified frames
modified_frames = bytes(frame_bytes)

In [12]:
# Store the final audio file
final_audio = wave.open("Encrypted.wav","wb")
final_audio.setparams(audio.getparams())
final_audio.writeframes(modified_frames)

### Decryption

In [13]:
# Load the final audio file and extract frames from it
audio = wave.open("Encrypted.wav", mode="rb")
frame_data = audio.readframes(audio.getnframes())
frame_bytes = bytearray(list(frame_data))

In [14]:
# Extract bits from the message and get number of frames from data
extracted = [frame_bytes[i] & 1 for i in range(len(frame_bytes[:word_size]))]
number_elements = int(str("".join(list(map(str,extracted)))),2)

In [15]:
# Get audio file size and divide it into sample sizes
frame_bytes_length = len(frame_bytes)-word_size
sample_size = frame_bytes_length//(number_elements*word_size)

In [16]:
# Extract frame data
finalExtraction = [] # Final extracted frame data
for i in range(0,(number_elements*word_size)):
    temp_frame_data = frame_bytes[word_size+i*sample_size:word_size+(i+1)*sample_size] # Audio data sample
    
    parity = 0 # Calcuate the xor of whole number
    for frame in temp_frame_data:
        parity = parity^frame
        
    final_parity = 0 # Get final parity which is the answer
    for parity_bit in '{0:08b}'.format(parity):
        final_parity = final_parity^int(parity_bit)
    finalExtraction.append(final_parity)

In [17]:
# Get original ordering of the bits
random_position = np.random.RandomState(seed=number_element*secret_key).permutation(number_element*word_size)
finalBits = []
for i in random_position:
    finalBits.append(finalExtraction[i])

In [18]:
# Extract bits and club them together to form word and then convert it to numbers
frame_list = list(np.array(finalBits).reshape(number_elements,word_size))
number = list(map(lambda x: int("".join(list(map(str,x))),2),frame_list))

In [19]:
print(number)

[165, 175, 200, 1000]
